# Enrichment of Weather Data
We now merge the weather data from meteostat with our regions.
(An upcoming project will be a comparison of meteostat vs open-meteo!)

In [27]:
#!conda env export > environment.yml
#!pip freeze | grep -v "^-e" | grep -v "@" | awk -F= '{print $1 "==" $3}' > requirements.txt

In [69]:
import numpy as np 
import pandas as pd
from geopy.exc import GeocoderTimedOut 
from geopy.geocoders import Nominatim 

from datetime import datetime
from meteostat import  Daily, Stations

import time

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [70]:
# function to find the coordinate of a given city  
def findGeocode(city): 
       
    # try and catch is used to overcome the exception thrown by geolocator 
    try:  
        geolocator = Nominatim(user_agent="data_coon") 
        time.sleep(1)
        return geolocator.geocode(city, language='en') 
      
    except GeocoderTimedOut: 
        time.sleep(2)
          
        return findGeocode(city)     


In [71]:
# make def to return latitude and longitude for a city given a country (more reliable because sometimes the same city name exists in different countries)
def get_lat_long(city, country):
    if findGeocode(city) != None: 
        if findGeocode(city).raw["display_name"].split(", ")[-1] == country:
            return findGeocode(city).latitude, findGeocode(city).longitude
        #return findGeocode(city).latitude, findGeocode(city).longitude
        else:
            # open csv file and append the data city to clean it later
            with open('log.csv', 'a') as f:
                # format: City: "city", Country: "country", Date: "date" (only date, not time)
                f.write("City: " + city + ", Country: " + country + ", Date: " +str(datetime.today().strftime('%d-%m-%Y')) + '\n')
            return np.nan, np.nan
    else:
        # open csv file and append the data city to clean it later
        with open('log.csv', 'a') as f:
            # format: City: "city", Country: "country", Date: "date" (only date, not time)
            f.write("City: " + city + ", Country: " + country + ", Date: " +str(datetime.today().strftime('%d-%m-%Y')) + '\n')
        return np.nan, np.nan

we also average the $n$ nearest stations for a more robust and denser data set.

In [72]:
# average a df per column based on a multiindex if its non empty
def average_df(df):
    if not df.empty:
        df = df.groupby(level=1).mean()
    return df

# make def to get data based on a city and year from the n nearest stations
def get_weather(lat, lon, year, n):
    start = datetime(int(year), 1, 1)
    end = datetime(int(year), 12, 31)

    #lat, lon = get_lat_long(city,country)
        # check if not nan
    #if np.isnan(lat) or np.isnan(lon):
    #    # break if nan
    #    return np.nan
    #else:
    stations = Stations()
    stations = stations.nearby(lat, lon)
    station = stations.fetch(n)
    data = Daily(station, start, end)
    data = data.fetch()
    return average_df(data)

### Reduce to growth period

In [73]:
# get df for certain time frame based on index
def get_growth_period(df, start, end):
    return df.loc[start:end]
#Growth period: may 11th - september 20th 
#https://en.wikipedia.org/wiki/Harvest_(wine)
#df = get_growth_period(df, '2019-03-11', '2019-09-20')

### get volatility

In [74]:
# get volatility of a single column
def get_volatility(df, column):
    return df[column].std()/df[column].mean()


### Longest drought

In [75]:
# get the longest consecutive sequence of a value in a df
def longest_sequence(df, column, value):
    return df[column].eq(value).astype(int).groupby(df[column].ne(value).cumsum()).sum().max()


### Longest rain period

In [76]:
# get the longest consecutive sequence of the absence (!) of a value in a df
# this is the longest sequence of days with rain
def longest_sequence_no(df, column, value):
    return df[column].ne(value).astype(int).groupby(df[column].eq(value).cumsum()).sum().max()

### Avg Rain

In [77]:
#Vines need between 400 and 600 mm of rain per year. 
#A regular supply of water throughout the growth cycle is needed for a high quality crop.
#https://www.idealwine.info/conditions-necessary-great-wine-part-12/
# get the avg of the column prcp for a df
def get_avg_prcp(df):
    return df['prcp'].mean()

### No of Days above 35 degrees 

In [78]:
#At temperatures below 10°C and above 35°C, photosynthesis will be disrupted and vines will not grow properly.
# count number of rows in a df column tmax above a threshold
def count_above(df, column, threshold):
    return df[df[column] > threshold][column].count()

### number of days below 10 degrees


In [79]:
# count number of rows in a df column tmax under a threshold

def count_under(df, column, threshold):
    return df[df[column] < threshold][column].count()

### coulure wspd

In [80]:
#strong wind around june --> coulure 
# get the avg of the column wspd for a df
def get_avg_wspd(df):
    year = df.index[0].year
    df = get_growth_period(df, f'{year}-05-15', f'{year}-07-15')
    return df['wspd'].mean()

### May July Rain

In [81]:
#Too much rain during the May-July period --> diseases such as mildew or oidium
# sum the column prcp for a df
def get_sum_prcp(df):
     year = df.index[0].year
     df = get_growth_period(df, f'{year}-05-15', f'{year}-07-15')
     return df['prcp'].sum()

## Combined Features
```
Temp Vola: get_volatility(df, 'tavg')
Rain Vola: get_volatility(df, 'prcp')
Longest drought: longest_sequence(df, 'prcp', 0)
Longest Rain: longest_sequence_no(df, 'prcp', 0) 
Avg Rain: get_avg_prcp(df)
NoDays35: count_above(df, 'tmax', 35)
NoDays10: count_under(df, 'tmin', 10)
NoDays0: count_under(df, 'tmin', 0)
ColourWspd: get_avg_wspd(df)
MayJulyRain: get_sum_prcp(df)
```

In [82]:
# combine all function in one function

def get_weather_features(lat, lon, year, n):
    raw = get_weather(lat, lon, year, n)
    if raw is np.nan:
        return pd.Series([np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan])
    else:
        df = get_growth_period(raw, f'{year}-03-11', f'{year}-09-20')
        return pd.Series([get_volatility(df, 'tavg'), get_volatility(df, 'prcp'), longest_sequence(df, 'prcp', 0), longest_sequence_no(df, 'prcp', 0), get_avg_prcp(df), count_above(df, 'tmax', 35), count_under(df, 'tmin', 10), count_under(df, 'tmin', 0), get_avg_wspd(df), get_sum_prcp(df)])

# Merge

In [83]:
# load drafts/df_out.csv
df = pd.read_csv('data/wine_data.csv')

In [84]:
df.head()

,Vintage,Score,Drink Window,Description,Country,Region,Variety
0,2021,93–97,NYR,As vines struggled to ripen their fruit at the...,United States,Napa,Cabernet
1,2020,87,Drink,The setup—wet winter into dry spring—was ideal...,United States,Napa,Cabernet
2,2019,97,Hold,A wet spring resulted in less overt tannic str...,United States,Napa,Cabernet
3,2018,99,Hold,A wet winter provided sufficient water through...,United States,Napa,Cabernet
4,2017,92,Drink or hold,"Drought broke over the winter, with lots of ve...",United States,Napa,Cabernet


In [85]:
# unfortunately some data is clean but the geopy API doesnt recognize it, so we adjust three more regions
#Barossa and McLaren Vale, Australia --> Adelaide
#Côtes de Nuits, France -> Vougeot
#Douro Valley, Portugal --> Coimbra

df.loc[(df['Region'] == 'Barossa and McLaren Vale'), 'Region'] = 'Adelaide'
df.loc[(df['Region'] == 'Côtes de Nuits'), 'Region'] = 'Vougeot'
df.loc[(df['Region'] == 'Douro Valley'), 'Region'] = 'Coimbra'


## Coordinates (lat, long)

In [86]:
# unique country + region tuples
unique = df[['Country', 'Region']].drop_duplicates()
unique

,Country,Region
0,United States,Napa
31,United States,Carneros Valley
57,United States,California
83,United States,Oregon
107,United States,Washington
152,Argentina,Mendoza
197,Australia,Adelaide
219,Australia,Victoria
259,Austria,Austria
317,Chile,Chile


In [87]:
# def get_lat_long(city, country):
unique[["Latitude", "Longitude"]] = unique.apply(lambda row: get_lat_long(row['Region'], row['Country']), axis=1)
unique


## Weather

In [67]:
# based on lat, long and vintage, get weather features

,Country,Region
0,United States,Napa
31,United States,Carneros Valley
57,United States,California
83,United States,Oregon
107,United States,Washington
152,Argentina,Mendoza
197,Australia,Adelaide
219,Australia,Victoria
259,Austria,Austria
317,Chile,Chile


In [68]:
df[["Vola_Temp", "Vola_Rain", "Longest_Dry", "Longest_Wet", "Avg_Rain", "Count_above35", "Count_under10", "Count_under0", "Coulure_Wind", "June_Rain"]] = df.apply(lambda row: get_weather_features(row['Region'], row['Country'], row['Vintage'], 5), axis=1)
df

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Oregon&format=json&limit=1&accept-language=en (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [ ]:
# save as CSV file
df.to_csv('data/merged_data.csv', index=False)